In [8]:
import pdfplumber
import re
import json
import os
import html
import PyPDF2
import pandas as pd


from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

## METODI

In [9]:
load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [10]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

### METODI

In [11]:
def carica_pianeti_e_distanze(file_csv: str):
    """
    Carica i pianeti come nodi e le distanze tra di loro come relazioni nel database Neo4j.
    Le relazioni di distanza vengono create in entrambe le direzioni.
    
    Args:
        file_csv (str): Percorso del file CSV contenente le distanze tra i pianeti.
    """
    
    # Carica il file CSV usando pandas
    df = pd.read_csv(file_csv, index_col=0)
    
    with driver.session() as session:
        # Crea i nodi per i pianeti
        pianeti = df.columns.tolist()  # Estrae i nomi dei pianeti
        
        for pianeta in pianeti:
            # Crea un nodo per ciascun pianeta
            query = "MERGE (p:Pianeta {nome: $nome})"
            session.run(query, nome=pianeta)
        
        # Crea le relazioni DISTANZA tra i pianeti in entrambe le direzioni
        for i, pianeta1 in enumerate(pianeti):
            for j, pianeta2 in enumerate(pianeti):
                if i < j:  # Evita di elaborare due volte la stessa coppia di pianeti
                    distanza = df.at[pianeta1, pianeta2]
                    
                    # Crea relazioni di distanza in entrambe le direzioni
                    query = """
                    MATCH (p1:Pianeta {nome: $pianeta1}), (p2:Pianeta {nome: $pianeta2})
                    MERGE (p1)-[:DISTANZA {km: $distanza}]->(p2)
                    MERGE (p2)-[:DISTANZA {km: $distanza}]->(p1)
                    """
                    session.run(query, pianeta1=pianeta1, pianeta2=pianeta2, distanza=distanza)
        
        print("Pianeti e distanze caricate correttamente in Neo4j in entrambe le direzioni.")

In [12]:

def carica_piatti(file_json: str):
    """
    Carica i piatti e i numeri associati come nodi nel database Neo4j.
    
    Args:
        file_json (str): Percorso del file JSON contenente i piatti e i numeri.
        uri (str): URI del database Neo4j.
        user (str): Nome utente per Neo4j.
        password (str): Password per Neo4j.
    """
    
    # Carica il file JSON
    with open(file_json, 'r', encoding='utf-8') as f:
        piatti_data = json.load(f)
    
    with driver.session() as session:
        for piatto, numero in piatti_data.items():
            # Decodifica i caratteri speciali (ad esempio per trattare le sequenze \u00e0)
            piatto_decodificato = html.unescape(piatto)
            
            # Crea un nodo per ciascun piatto
            query = """
            MERGE (p:Piatto {nome: $nome})
            SET p.numero = $numero
            """
            session.run(query, nome=piatto_decodificato, numero=numero)
        
        print("Piatti caricati correttamente in Neo4j.")

In [ ]:
def update_piatti(file_json: str, llm_client):
    """
    Carica i piatti dal JSON e li merge con quelli esistenti in Neo4j,
    gestendo differenze nei nomi e rimuovendo piatti senza numero.
    
    Args:
        file_json (str): Percorso del file JSON contenente i piatti e i numeri.
        llm_client: Client del modello LLM (OpenAI o1-mini) per matching dei nomi.
    """
    
    # Carica il file JSON
    with open(file_json, 'r', encoding='utf-8') as f:
        piatti_data = json.load(f)
    
    # Decodifica i caratteri speciali nei nomi dei piatti
    piatti_decodificati = {html.unescape(piatto): numero for piatto, numero in piatti_data.items()}
    
    with driver.session() as session:
        # 1. Scarica tutti i piatti esistenti
        query_get_piatti = "MATCH (p:Piatto) RETURN p.nome as nome"
        result = session.run(query_get_piatti)
        piatti_esistenti = [record["nome"] for record in result]
        
        # 2. Trova corrispondenze tra piatti nuovi e esistenti
        corrispondenze = {}
        piatti_senza_corrispondenza = []
        
        for piatto_nuovo, numero in piatti_decodificati.items():
            migliore_corrispondenza = None
            max_score = 0
            
            if piatto_nuovo in piatti_esistenti:
                # Match esatto
                corrispondenze[piatto_nuovo] = piatto_nuovo
                continue
                
            # Usa l'LLM per trovare la migliore corrispondenza
            candidates = []
            for piatto_esistente in piatti_esistenti:
                # Pre-filtraggio semplice per ridurre il carico sul LLM
                if len(piatto_nuovo) > 5 and len(piatto_esistente) > 5:
                    # Verifica se ci sono parole comuni
                    parole_nuove = set(piatto_nuovo.lower().split())
                    parole_esistenti = set(piatto_esistente.lower().split())
                    if parole_nuove.intersection(parole_esistenti):
                        candidates.append(piatto_esistente)
            
            if candidates:
                # Usiamo l'LLM per la corrispondenza
                prompt = f"""
                Devo trovare corrispondenze tra nomi di piatti con piccole differenze di scrittura.
                Piatto da cercare: "{piatto_nuovo}"
                Candidati: {candidates}
                
                Indica il nome tra i candidati che corrisponde al piatto da cercare, 
                considerando differenze come spazi, trattini, maiuscole/minuscole, o piccoli errori di battitura.
                Rispondi solo con il nome esatto del candidato, o "Nessuna corrispondenza" se non ce ne sono.
                """
                
                risposta = llm_client.generate_text(prompt)
                
                if risposta.strip() != "Nessuna corrispondenza" and risposta.strip() in candidates:
                    corrispondenze[piatto_nuovo] = risposta.strip()
                else:
                    piatti_senza_corrispondenza.append(piatto_nuovo)
            else:
                piatti_senza_corrispondenza.append(piatto_nuovo)
        
        # 3. Aggiorna i piatti esistenti e crea quelli nuovi
        for piatto_nuovo, numero in piatti_decodificati.items():
            if piatto_nuovo in corrispondenze:
                # Aggiorna il piatto esistente
                piatto_esistente = corrispondenze[piatto_nuovo]
                query = """
                MATCH (p:Piatto {nome: $nome_esistente})
                SET p.numero = $numero, p.nome_alternativo = $nome_nuovo
                """
                session.run(query, nome_esistente=piatto_esistente, 
                           nome_nuovo=piatto_nuovo, numero=numero)
            else:
                # Crea un nuovo piatto
                query = """
                CREATE (p:Piatto {nome: $nome, numero: $numero})
                """
                session.run(query, nome=piatto_nuovo, numero=numero)
        
        # 4. Elimina i piatti senza numero
        query_delete = """
        MATCH (p:Piatto)
        WHERE p.numero IS NULL
        DELETE p
        """
        result = session.run(query_delete)
        num_deleted = result.consume().counters.nodes_deleted
        
        print(f"Piatti aggiornati: {len(corrispondenze)}")
        print(f"Piatti nuovi creati: {len(piatti_senza_corrispondenza)}")
        print(f"Piatti eliminati (senza numero): {num_deleted}")
        print("Operazione completata.")

## MAIN

In [13]:
# Esegui il codice passando il percorso del file CSV e le credenziali di Neo4j
file_csv = '../Hackapizza Dataset/Misc/Distanze.csv'  # Percorso al tuo file CSV

carica_pianeti_e_distanze(file_csv)


Pianeti e distanze caricate correttamente in Neo4j in entrambe le direzioni.


In [14]:
# Esegui il codice passando il percorso del file JSON e le credenziali di Neo4j
file_json = '../Hackapizza Dataset/Misc/dish_mapping.json'  # Percorso al tuo file JSON

carica_piatti(file_json)


Piatti caricati correttamente in Neo4j.
